- 댓글에 대한 혐오 유무도 있지만 nsmc와 감성분석 측면에서 겹치므로 사회적 편견에 대한 유무만 평가

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from vllm import LLM, SamplingParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import glob

/home/aicombined/miniconda3/envs/poc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-17 21:03:51,688	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


- Instruction Formatting 전처리 함수

In [2]:
def preprocess_text(x):
    return f"###입력:{x}\n\n###출력:"

- merge_16bit로 저장된 모델 weight의 폴더를 이용하여 llm를 로드

In [3]:
llm = LLM(model="huggingface_merged_weights/SOLAR/", enable_lora=True)

INFO 06-17 21:03:52 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='huggingface_merged_weights/SOLAR/', speculative_config=None, tokenizer='huggingface_merged_weights/SOLAR/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=huggingface_merged_weights/SOLAR/)
INFO 06-17 21:03:52 utils.py:660] Found nccl from library /home/aicombined/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-17 21:03:53 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-17 21:03:53 sele

- 자연어 생성을 위한 그리드 서치 디코딩 파라미터 선언

In [4]:
sampling_params = SamplingParams(max_tokens=256, temperature=0.5, top_k=50)

- korea_hate_speech 데이터셋 불러오기

In [5]:
datapaths = glob.glob('./evaluation_dataset/02_korea_hate_speech/*.tsv')
sentences = []
labels = []

for path in datapaths:
    with open(path, 'r') as f:
        hate_data = f.readlines()
            
    for data in hate_data[1:]:
        data_split = data.split('\t')
        sentence = data_split[0]
        social_bias = data_split[2]
    
        sentences.append(sentence)
    
        if social_bias == 'none': 
            labels.append("사회적 편견 없음")
        else:
            labels.append("사회적 편견 있음")

In [7]:
len(sentences), len(labels)

(8367, 8367)

- 입력 데이터 전처리

In [8]:
system_prompt = """사회적 편견은 특정한 개인이나 집단에 대해 가지는 고정된, 비합리적이며 부정확한 믿음이나 태도를 의미합니다. 문장에 대하여 사회적 편견 있음 또는 사회적 편견 없음으로 대답하세요.\n\n"""
texts = []

for sentence, label in zip(sentences, labels):
    instruction = sentence
    texts.append(preprocess_text(system_prompt + instruction))

- inference

In [9]:
outputs = llm.generate(texts, sampling_params)

Processed prompts: 100%|██████████| 8367/8367 [06:31<00:00, 21.38it/s] 


- 생성 결과 전처리

In [10]:
preds = []
for output in outputs:
    generated_text = output.outputs[0].text
    preds.append(generated_text.split('$&%')[0])

In [11]:
# prediction 값이 사회적 편견 없음 또는 사회적 편견 있음이 아니면 틀린것으로 간주, 정답 데이터와 반대 되는 label를 삽입
preds_processed = []
labels_processed = []

for y_pred, y_true in zip(preds, labels):
    if y_pred == '사회적 편견 없음' or y_pred == '사회적 편견 있음':
        preds_processed.append(y_pred)
        labels_processed.append(y_true)

- zero-shot 분류 성능

In [12]:
print(classification_report(labels_processed, preds_processed))

              precision    recall  f1-score   support

   사회적 편견 없음       0.79      0.60      0.69      5276
   사회적 편견 있음       0.48      0.70      0.57      2789

    accuracy                           0.64      8065
   macro avg       0.64      0.65      0.63      8065
weighted avg       0.68      0.64      0.65      8065

